# ID Compare · 4 Fields (Batch)

Notebook de démonstration : OCR des cartes d'identité → comparaison avec Excel.

**Entrées**: images anonymisées dans `examples/sample_ids/` et `examples/sample_registry.xlsx`.
**Sortie**: `examples/expected_output.csv`.


In [ ]:
IMAGES_DIR = "examples/sample_ids"  # dossier d'images (anonymisées)\nEXCEL_PATH = "examples/sample_registry.xlsx"\nOUTPUT_CSV = "examples/expected_output.csv"\nLANG = "fra"\nTHRESH = 127\nFUZZY_THRESHOLD = 85\n

In [ ]:
import pandas as pd, numpy as np, os, glob\nfrom src.id_compare.ocr import read_text\nfrom src.id_compare.match import score_pair\nfrom src.id_compare.utils import normalize_name, normalize_dob, parse_id\n\ndf_registry = pd.read_excel(EXCEL_PATH)\nrows = []\nfor img_path in sorted(glob.glob(os.path.join(IMAGES_DIR, '*'))):\n    text = ""  # Remplacer par read_text(img_path, lang=LANG) si images présentes\n    rid = parse_id(text)\n    # Démo: on simule un mapping image→ligne si noms de fichiers ID_001/002... existent\n    base = os.path.basename(img_path)\n    if base.startswith('ID_001'):\n        rid, dob, ln, fn = 'GN123456', '1993-05-21', 'CAMARA', 'ALPHA OUMAR'\n    elif base.startswith('ID_002'):\n        rid, dob, ln, fn = 'GN654321', '1988-09-09', 'DIOP', 'AMADOU OURY'\n    else:\n        rid, dob, ln, fn = None, None, None, None\n    # Recherche registre\n    rec = df_registry[df_registry['id_number']==rid].head(1).to_dict('records')\n    rec = rec[0] if rec else {{'id_number':None,'dob':None,'last_name':None,'first_name':None}}\n    score = int(np.mean([\n        score_pair(rid, rec['id_number']),\n        score_pair(dob, str(rec['dob'])),\n        score_pair(ln, rec['last_name']),\n        score_pair(fn, rec['first_name']),\n    ]))\n    rows.append({{\n        'image_file': base,\n        'id_match': rid == rec['id_number'],\n        'dob_match': dob == str(rec['dob']),\n        'last_name_match': normalize_name(ln)==normalize_name(rec['last_name']),\n        'first_name_match': normalize_name(fn)==normalize_name(rec['first_name']),\n        'score': score\n    }})\nout = pd.DataFrame(rows)\nout.to_csv(OUTPUT_CSV, index=False)\nout.head()\n